In [6]:
from pyadjoint import Block, annotate_tape, stop_annotating
from fenics_adjoint.types import create_overloaded_object
import fenics as df
import fenics_adjoint as dfa
import pyadjoint as pya

def normalise(func):
    vec = func.vector()
    vec /= vec.norm('l2')
    return dfa.Function(func.function_space(), vec)

backend_normalise = normalise

In [7]:
mesh = df.UnitSquareMesh(10, 10)
V = dfa.FunctionSpace(mesh, 'CG', 1)

f = dfa.project(dfa.Expression('x[0]*x[1]', degree=1), V)

g = normalise(f)

J = dfa.assemble(g*df.dx)

In [8]:
def normalise(func, **kwargs):
    annotate = annotate_tape(kwargs)

    if annotate:
        tape = dfa.get_working_tape()
        block = NormaliseBlock(func)
        tape.add_block(block)

    with stop_annotating():
        output = backend_normalise(func, **kwargs)

    output = create_overloaded_object(output)

    if annotate:
        block.add_output(output.create_block_variable())

    return output

In [9]:
class NormaliseBlock(Block):
    def __init__(self, func, **kwargs):
        super(NormaliseBlock, self).__init__()
        self.kwargs = kwargs
        self.add_dependency(func.block_variable)

    def __str__(self):
        return 'NormaliseBlock'

    def evaluate_adj(self):
        
        adj_input = self.get_outputs()[0].adj_value
        dependency = self.get_dependencies()[0]
        x = dependency.saved_output.vector()

        adj_output = x.copy()

        xnorm = x.norm('l2')

        const = 0
        for i in range(len(x)):
            const += adj_input[i][0]*x[i][0]
        const /= xnorm**3

        for i in range(len(x)):
            adj_output[i] = adj_input[i][0]/xnorm - const*x[i][0]
        dependency.add_adj_output(adj_output)

    def recompute(self):
        dependencies = self.get_dependencies()
        func = dependencies[0].saved_output
        output = backend_normalise(func)
        self.get_outputs()[0].checkpoint = output
        

In [10]:
mesh = df.UnitSquareMesh(10, 10)
V = dfa.FunctionSpace(mesh, 'CG', 1)

f = dfa.project(dfa.Expression('x[0]*x[1]', degree=1), V)

g = normalise(f)

J = df.assemble(g*df.dx)

h = df.Function(V)
h.vector()[:] = 0.1
pya.ReducedFunctional(J, dfa.Control(f))
pya.taylor_test(pya.ReducedFunctional(J, dfa.Control(f)), f, h)

AttributeError: 'float' object has no attribute 'block_variable'

In [11]:
pya.ReducedFunctional(J, dfa.Control(f))